In [24]:
import math
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.svm import SVR
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
from sklearn.neighbors import KNeighborsRegressor
from sklearn.preprocessing import MinMaxScaler, StandardScaler, LabelEncoder
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

In [25]:
SEGMENT = 1

In [26]:
df = pd.read_csv(f'../dataset/preprocessed-training-data/power-laws-forecasting-energy-consumption-preprocessed-training-data-split_{SEGMENT}.csv')
df

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Temperature_4,Distance_1,Distance_2,Distance_3,Distance_4,DateInWeek,Time,Hour,TimeInterval,PartOfDay
0,0,143,2017-08-12 20:30:00+00:00,3145,289058.731029,65578.168728,15.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Saturday,20:30:00,20,20-21,Evening
1,1,300,2017-07-28 14:45:00+00:00,6718,37873.175383,3813.660202,15.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Friday,14:45:00,14,14-15,Afternoon
2,5,201,2014-07-24 12:30:00+00:00,4134,97.720896,11990.293968,15.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Thursday,12:30:00,12,12-13,Morning
3,6,271,2016-01-24 18:40:00+00:00,5926,133.267974,414.211270,5.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Sunday,18:40:00,18,18-19,Afternoon
4,7,276,2015-12-11 19:05:00+00:00,6065,11391.730143,621.276731,5.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Friday,19:05:00,19,19-20,Evening
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,121730,143,2017-07-28 20:15:00+00:00,3144,911229.420901,65578.168728,15.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Friday,20:15:00,20,20-21,Evening
99996,121731,140,2016-06-02 16:00:00+00:00,3005,65610.789794,16355.878746,15.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Thursday,16:00:00,16,16-17,Afternoon
99997,121732,259,2016-03-04 07:30:00+00:00,5399,15981.495896,22445.640302,30.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Friday,07:30:00,7,7-8,Morning
99998,121733,22,2017-06-23 14:40:00+00:00,734,48704.695637,7392.365415,10.0,18.0,False,False,...,9.178482,0.0,0.0,0.0,0.0,Friday,14:40:00,14,14-15,Afternoon


In [27]:
df = pd.get_dummies(df, columns=['DateInWeek', 'Holiday'])
def label_encoding(x):
    help = {
        'Monday':2,
        'Tuesday':3,
        'Wednesday': 4,
        'Thursday': 5,
        'Friday': 6,
        'Saturday': 7,
        'Sunday': 1,
        'Morning': 0,
        'Afternoon': 1,
        'Evening': 2,
        'Night': 3
    }
    return help[x]

df['PartOfDay'] = df['PartOfDay'].apply(label_encoding)

df

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Hour,TimeInterval,PartOfDay,DateInWeek_Friday,DateInWeek_Monday,DateInWeek_Saturday,DateInWeek_Sunday,DateInWeek_Thursday,DateInWeek_Tuesday,DateInWeek_Wednesday
0,0,143,2017-08-12 20:30:00+00:00,3145,289058.731029,65578.168728,15.0,18.0,False,False,...,20,20-21,2,0,0,1,0,0,0,0
1,1,300,2017-07-28 14:45:00+00:00,6718,37873.175383,3813.660202,15.0,18.0,False,False,...,14,14-15,1,1,0,0,0,0,0,0
2,5,201,2014-07-24 12:30:00+00:00,4134,97.720896,11990.293968,15.0,18.0,False,False,...,12,12-13,0,0,0,0,0,1,0,0
3,6,271,2016-01-24 18:40:00+00:00,5926,133.267974,414.211270,5.0,18.0,False,False,...,18,18-19,1,0,0,0,1,0,0,0
4,7,276,2015-12-11 19:05:00+00:00,6065,11391.730143,621.276731,5.0,18.0,False,False,...,19,19-20,2,1,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,121730,143,2017-07-28 20:15:00+00:00,3144,911229.420901,65578.168728,15.0,18.0,False,False,...,20,20-21,2,1,0,0,0,0,0,0
99996,121731,140,2016-06-02 16:00:00+00:00,3005,65610.789794,16355.878746,15.0,18.0,False,False,...,16,16-17,1,0,0,0,0,1,0,0
99997,121732,259,2016-03-04 07:30:00+00:00,5399,15981.495896,22445.640302,30.0,18.0,False,False,...,7,7-8,0,1,0,0,0,0,0,0
99998,121733,22,2017-06-23 14:40:00+00:00,734,48704.695637,7392.365415,10.0,18.0,False,False,...,14,14-15,1,1,0,0,0,0,0,0


In [28]:
attributes = [i for i in range(1, 32) if i not in [2, 3, 4, 16, 25, 27]]
training_data = df.values
X_train = training_data[:, attributes]
y_train = training_data[:, 4]
X_train
# df.columns

,obs_id,SiteId,Timestamp,ForecastId,Value,Surface,Sampling,BaseTemperature,MondayIsDayOff,TuesdayIsDayOff,...,Hour,TimeInterval,PartOfDay,DateInWeek_Friday,DateInWeek_Monday,DateInWeek_Saturday,DateInWeek_Sunday,DateInWeek_Thursday,DateInWeek_Tuesday,DateInWeek_Wednesday


In [29]:
svm_regressor = SVR(kernel='rbf', C=10, epsilon=24)
rf_regressor = RandomForestRegressor(n_estimators=200, random_state=24)
grdb_regressor = GradientBoostingRegressor(n_estimators=500, max_depth=4, min_samples_split=5, learning_rate=0.01, loss='squared_error')

svm_regressor.fit(X_train, y_train)
rf_regressor.fit(X_train, y_train)
grdb_regressor.fit(X_train, y_train)

y_pred_svm = svm_regressor.predict(X_test)
y_pred_rf = rf_regressor.predict(X_test)
y_pred_grdb = grdb_regressor.predict(X_test)

mae_svm = mean_absolute_error(y_test, y_pred_svm)
mae_rf = mean_absolute_error(y_test, y_pred_rf)
mae_grdb = mean_absolute_error(y_test, y_pred_grdb)

mse_svm = mean_squared_error(y_test, y_pred_svm)
mse_rf = mean_squared_error(y_test, y_pred_rf)
mse_grdb = mean_squared_error(y_test, y_pred_grdb)

rmse_svm = math.sqrt(mse_svm)
rmse_rf = math.sqrt(mse_rf)
rmse_grdb = math.sqrt(mse_grdb)

r2_svm = r2_score(y_test, y_pred_svm)
r2_rf = r2_score(y_test, y_pred_rf)
r2_grdb = r2_score(y_test, y_pred_grdb)
